find the reason of which,how and why certain categories seems to be trending and why certain categories are not, because we want to predict the trending categories, explore everything, and create a column which defines whether a category is trending (1) or not (0) binary classification

##########################

'video_id', 'trending_date', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed',
'description', 'category', 'sum_stats', 'trending'


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import datetime
from datetime import date

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("whitegrid")

# import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

cf.go_offline()

import pandas_profiling
import plotly.graph_objects as go

from wordcloud import WordCloud,STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
df = pd.read_csv('../input/youtubedata/UStrends.csv')
df.rename(columns = {'trend':'trending'}, inplace = True)
df.drop(['Unnamed: 0.1','thumbnail_link','video_id'],axis=1,inplace=True)
df.head(5)
# trend of a category was calculated by summing the views, likes and comment_count minus by dislikes, sum_stats 
# column was created with result of these calculations then sum_stats mean was taken and the values greater than
# that mean is considered trending (1) and values less than that mean are considered as non trending (0). 

In [ ]:
px.line(df,x='trending_date',y=['views','likes','comment_count'],labels={'value':'sum of stats'})
# plotting line graph of views, likes and comment_count over their trending date

In [ ]:
px.line(df,x='publish_time',y=['views','likes','dislikes','comment_count'])
# plotting line graph of views, likes, dislikes and comment_count over their publish time 

In [ ]:
px.line(df,x='trending_date',y='likes')
# plotting line graph of likes over it's trending date

In [ ]:
px.line(df,x='trending_date',y='dislikes')
# plotting line graph of dislikes over it's trending date

In [ ]:
px.line(df,x='trending_date',y='comment_count')
# plotting line graph of comment_count over it's trending date

In [ ]:
cat1=df.groupby('category')[['likes','dislikes','comment_count']].sum().sort_values(by='likes',ascending=False)
px.bar(cat1,barmode='group')
# likes, dislikes and comment_count for each category

In [ ]:
cat2=df.groupby('category')['views'].agg(['sum']).sort_values(by='sum',ascending=False)
px.bar(cat2,barmode='group',labels={'value':'views'})
# views for each category

In [ ]:
cat3=df.groupby('comments_disabled')[['views','likes','dislikes','comment_count']].sum().sort_values(by='likes',ascending=False)
px.bar(cat3,barmode='group')
# 'views','likes','dislikes','comment_count' for each comment disabled

In [ ]:
cat4=df.groupby('ratings_disabled')[['views','likes','dislikes','comment_count']].sum().sort_values(by='likes',ascending=False)
px.bar(cat4,barmode='group')
# 'views','likes','dislikes','comment_count' for each ratings_disabled

In [ ]:
cat5=df.groupby('video_error_or_removed')[['views','likes','dislikes','comment_count']].sum().sort_values(by='likes',ascending=False)
px.bar(cat5,barmode='group')
# 'views','likes','dislikes','comment_count' for each video_error_or_removed

In [ ]:
cat5=df.groupby('trending')[['views','likes','dislikes','comment_count']].sum().sort_values(by='likes',ascending=False)
px.bar(cat5,barmode='group')
# # 'views','likes','dislikes','comment_count' for each trending

In [ ]:
px.bar(df.groupby('category',as_index=False)['trending'].agg(['sum']).sort_values(by='sum',ascending=False))
# trend of cateogries

In [ ]:
# set(df['category'])
cat_t=df.groupby('category')['trending'].value_counts()
cat_t
# number of trend and non trend for each category 

In [ ]:
df['comments_disabled']=np.where(df['comments_disabled']==False,0,1)
df['ratings_disabled']=np.where(df['ratings_disabled']==False ,0,1)
df['video_error_or_removed']=np.where(df['video_error_or_removed']==False ,0,1)
df

In [ ]:
trend_1=df[df['trending']==1]
trend_1=trend_1.reset_index(drop=True)
trend_0=df[df['trending']==0]
trend_0=trend_0.reset_index(drop=True)
trend_1.head(5)

In [ ]:
stopwords = set(STOPWORDS)
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='black',
        stopwords=stopwords,
        max_words=200000,
        max_font_size=40, 
        scale=3,
        random_state=1).generate(str(data))
    
    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)
        
    plt.imshow(wordcloud)
    plt.show()
print("Title - Trend 1")
show_wordcloud(trend_1['title'].values)
print("Title - Trend 0")
show_wordcloud(trend_0['title'].values)
# learn about the wordcloud from the internet
# generating wordcloud of the title column for all 1 for trending and 0 for non trending

In [ ]:
print("Tags - Trend 1")
show_wordcloud(trend_1['tags'].values)
print("Tags - Trend 0")
show_wordcloud(trend_0['tags'].values)
# generating wordcloud of the tags column for all 1 for trending and 0 for non trending

In [ ]:
print("description - Trend 1")
show_wordcloud(trend_1['description'].values)
print("description - Trend 0")
show_wordcloud(trend_0['description'].values)
# generating wordcloud of the description column for all 1 for trending and 0 for non trending

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.heatmap(df.corr(),annot=True,cmap='viridis', fmt='g')
# sum_stats, likes and views columns are partially correlated with trending column

1. Music , Entertainment and comedy have the highest likes, dislikes comment_count

2. Music , Entertainment and Film & Animation have the highest views.

3. trending videos have the most views compared to non trending

4. Music, Entertainment and Film & Animation are the most trending categories.

Starting the feature engineering

In [ ]:
fdf=df[['category','views','likes','dislikes','comment_count','sum_stats','trending']]
fdf
# getting the required columns and discarding the rest of the columns 

In [ ]:
fdf['music_f']=np.where((fdf['category']=='Music'),1,0)
# creating feature of music category since it is the most trending category

In [ ]:
fdf['views_f']=np.where(fdf['views']>fdf['views'].mean(),1,0)
fdf['likes_f']=np.where(fdf['likes']>fdf['likes'].mean(),1,0)
fdf['dislikes_f']=np.where(fdf['dislikes']>fdf['dislikes'].mean(),1,0)
fdf['cc_f']=np.where(fdf['comment_count']>fdf['comment_count'].mean(),1,0)

fdf
# since the videos being trending or not was decided based on the mean of sum_stats which was the sum of views 
# likes and comment_count and minus of dislikes, it is a good idea to feature engineering them the same way so that
# their correlation with trending column can get higher.

In [ ]:
fdf.drop(['category','views', 'likes' ,'dislikes' ,'comment_count' ,'sum_stats'],axis=1,inplace=True)
# we are done feature engineering with the columns, now we can drop all of them

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10 ,7)
sns.heatmap(fdf.corr(),annot=True,cmap='viridis', fmt='g')
# as you can see that resulting columns correlation have gotten alot better with the target column trending.

In [ ]:
fdf
# our resulting data look like this, we dropped the ss_f column because the our label column was originated from
# sum_stats so it's not right add such column to avoid overfitting since it was completely correlated 

In [ ]:
X2=fdf[['music_f' ,'likes_f' ,'dislikes_f' ,'cc_f']]
y2=fdf['trending']
# , 'views_f'
# we did not use the views_f column because it was causing the models to overfitt, we aim to create a more 
# generalized data for our models 
# separating the target or dependent column from the independent columns or features 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.3,random_state=42)
# splitting the data to xtrain , xtest and ytrain , ytest

Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
# training the Logistics regression model

In [ ]:
lr_pred = logreg.predict(X_test)
accuracy_score(lr_pred, y_test) * 100
# predicting the trend and calculating the accuracy

In [ ]:
cv_scores = cross_val_score(logreg,X2,y2,cv=5)
np.mean(cv_scores)*100
# calculating cross validation score of the dataset

In [ ]:
confusion_matrix(y_test,lr_pred)
# calculating confusion matrix 

In [ ]:
print(classification_report(y_test,lr_pred))
# complete performance metric report of the logistic regression

In [ ]:
lr = pd.DataFrame({'Actual': y_test, 'Predicted': lr_pred})
categories=[]
for i in lr.index:
    categories.append(df.iloc[i,15])

cats=pd.DataFrame(categories)
cats=cats.set_index(lr.index)

lr['categories']=cats

lr=lr.dropna(axis=0)

lr_trend=lr.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)

px.bar(lr_trend,barmode='group',labels={'value':'trend level'},title='Logistic Regression')
# visualizing the predictions 

In [ ]:
lr.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)
# data which our above visualization is based on.

Decision Tree Classifier

In [ ]:
decisiontree = DecisionTreeClassifier(random_state=0)
dep = np.arange(1,10)
param_grid = {'max_depth' : dep}

clf_cv = GridSearchCV(decisiontree, param_grid=param_grid, cv=5)

clf_cv.fit(X_train,y_train)

In [ ]:
dt_pred=clf_cv.predict(X_test)

clf_cv.best_params_,clf_cv.best_score_*100
print('Best value of max_depth:',clf_cv.best_params_)
# print('Accuracy:',clf_cv.best_score_*100)
accuracy_score(dt_pred, y_test) * 100

In [ ]:
cv_scores = cross_val_score(decisiontree,X2,y2,cv=5)
np.mean(cv_scores)*100

In [ ]:
confusion_matrix(y_test,dt_pred)

In [ ]:
print(classification_report(y_test,dt_pred))

In [ ]:
dt = pd.DataFrame({'Actual': y_test, 'Predicted': dt_pred})

# categories=[]
# for i in dt.index:
#     categories.append(df.iloc[i,15])

cats=pd.DataFrame(categories)
cats=cats.set_index(dt.index)

dt['categories']=cats

dt=dt.dropna(axis=0)

dt_trend=dt.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)

px.bar(dt_trend,barmode='group',labels={'value':'trend level'},title='Decision Tree Classifier')

In [ ]:
dt.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)

random forest classifier

In [ ]:
random_forest = RandomForestClassifier()
ne = np.arange(1,20)
param_grid = {'n_estimators' : ne}

rf_cv = GridSearchCV(random_forest, param_grid=param_grid, cv=5)

rf_cv.fit(X_train,y_train)

In [ ]:
rf_pred=rf_cv.predict(X_test)

rf_cv.best_params_,rf_cv.best_score_*100
print('Best value of max_depth:',rf_cv.best_params_)
accuracy_score(rf_pred, y_test) * 100

In [ ]:
cv_scores = cross_val_score(random_forest,X2,y2,cv=5)
np.mean(cv_scores)*100

In [ ]:
confusion_matrix(y_test,rf_pred)

In [ ]:
print(classification_report(y_test,rf_pred))

In [ ]:
rf = pd.DataFrame({'Actual': y_test, 'Predicted': rf_pred})

cats=pd.DataFrame(categories)
cats=cats.set_index(rf.index)

rf['categories']=cats

rf=rf.dropna(axis=0)

rf_trend=rf.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)

px.bar(rf_trend,barmode='group',labels={'value':'trend level'},title='Random Forest Classifier')

In [ ]:
rf.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)

Gradient boosting classfier

In [ ]:
gbk = GradientBoostingClassifier()
ne = np.arange(1,20)
dep = np.arange(1,10)
param_grid = {'n_estimators' : ne,'max_depth' : dep}

gbk_cv = GridSearchCV(gbk, param_grid=param_grid, cv=5)

gbk_cv.fit(X_train,y_train)

In [ ]:
gbk_pred=rf_cv.predict(X_test)

print('Best value of parameters:',gbk_cv.best_params_)
print('Best score:',gbk_cv.best_score_*100)

In [ ]:
cv_scores = cross_val_score(gbk,X2,y2,cv=5)
np.mean(cv_scores)*100

In [ ]:
confusion_matrix(y_test,gbk_pred)

In [ ]:
print(classification_report(y_test,gbk_pred))

In [ ]:
gbks = pd.DataFrame({'Actual': y_test, 'Predicted': gbk_pred})

cats=pd.DataFrame(categories)
cats=cats.set_index(gbks.index)

gbks['categories']=cats

gbks=gbks.dropna(axis=0)

gbks_trend=gbks.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)

px.bar(gbks_trend,barmode='group',labels={'value':'trend level'},title='Gradient Boosting Classifier')

In [ ]:
gbks.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)

K nearest neighbor

In [ ]:
knn = KNeighborsClassifier( )
k_range = list(range(1,10))
weights_options = ['uniform','distance']
k_grid = dict(n_neighbors=k_range, weights = weights_options)
grid = GridSearchCV(knn, k_grid, cv=10, scoring = 'precision')
grid.fit(X_train,y_train)

In [ ]:
knn_pred=grid.predict(X_test)
print('Best value of parameters:',grid.best_params_)
print('Best score:',grid.best_score_*100)

In [ ]:
cv_scores = cross_val_score(knn,X2,y2,cv=5)
np.mean(cv_scores)*100

In [ ]:
confusion_matrix(y_test,knn_pred)

In [ ]:
print(classification_report(y_test,knn_pred))

In [ ]:
knns = pd.DataFrame({'Actual': y_test, 'Predicted': knn_pred})

cats=pd.DataFrame(categories)
cats=cats.set_index(knns.index)

knns['categories']=cats

knns=knns.dropna(axis=0)

knns_trend=knns.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)

px.bar(knns_trend,barmode='group',labels={'value':'trend level'},title='K Nearest Neighbor Classifier')

In [ ]:
knns.groupby('categories')[['Predicted','Actual']].sum().sort_values(by='Predicted',ascending=False)